In [1]:
import pyspark

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/05 22:01:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
spark.version

'3.5.1'

In [13]:
# df = spark.read \
#     .option("header", "true") \
#     .csv('fhv_tripdata_2019-10.csv')

In [15]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropOff_datetime: string (nullable = true)
 |-- PUlocationID: string (nullable = true)
 |-- DOlocationID: string (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [16]:
df = df.repartition(6)
df.write.parquet('fhv_tripdata_october/')

In [17]:
df = spark.read.parquet('fhv_tripdata_october')

In [18]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B03016|2019-10-02 16:00:16|2019-10-02 16:36:20|         264|         174|   NULL|                B02969|
|              B03157|2019-10-01 18:23:49|2019-10-01 18:44:35|         264|         226|   NULL|                B03157|
|              B00821|2019-10-03 11:35:41|2019-10-03 11:43:43|         264|          76|   NULL|                B02872|
|              B02416|2019-10-02 10:37:00|2019-10-02 11:38:38|        NULL|        NULL|   NULL|                B02416|
|              B01061|2019-10-01 08:53:23|2019-10-01 09:02:42|         264|         126|   NULL|                B01061|
|              B01087|2019-10-03 03:41:5

In [6]:
from pyspark.sql import types

schema = types.StructType([
        types.StructField('dispatching_base_num', types.StringType(),True), 
        types.StructField('pickup_datetime', types.TimestampType(), True),
        types.StructField('dropOff_datetime', types.TimestampType(), True),
        types.StructField('PUlocationID',types. IntegerType(), True),
        types.StructField('DOlocationID',types. IntegerType(), True),
        types.StructField('SR_Flag',types. StringType(), True),
        types.StructField('Affiliated_base_number',types. StringType(), True)
                          ])

In [33]:
filtered_df = df.filter((df['pickup_datetime'] >= '2019-10-15 00:00:00') &
                        (df['pickup_datetime'] <= '2019-10-15 23:59:59'))

In [34]:
filtered_df.count()

62610

In [34]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhv_tripdata_2019-10.csv')

# df = spark.read \
#     .option("header", "true") \
#     .csv('fhv_tripdata_2019-10.csv')

In [47]:
from pyspark.sql.functions import col, unix_timestamp

# Assuming your DataFrame is named 'df' and you have columns 'start' and 'end'

# Convert timestamp columns to Unix timestamps (in seconds)
df = df.withColumn("start_unix", unix_timestamp(col("pickup_datetime")))
df.show()

df = df.withColumn("end_unix", unix_timestamp(col("dropOff_datetime")))
df.show()

# Calculate the difference between end and start timestamps
df = df.withColumn("duration_seconds", col("dropOff_datetime") - col("pickup_datetime"))

# Find the maximum duration
# max_duration = df.agg({"duration_seconds": "max"}).collect()[0][0]

# # Display the maximum duration
# print("Maximum duration (seconds):", max_duration)

# df.show()

# count_df = df.groupBy("PUlocationID").count()
count_df = df.orderBy(col('duration_seconds').desc())
pandas_df = count_df.select('duration_seconds').toPandas()

print(pandas_df)

# df_with_duration.printSchema()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+----------+----------+--------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|start_unix|  end_unix|    duration_seconds|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+----------+----------+--------------------+
|              B00009|2019-10-01 00:23:00|2019-10-01 00:35:00|         264|         264|   NULL|                B00009|1569914580|1569915300|INTERVAL '0 00:12...|
|              B00013|2019-10-01 00:11:29|2019-10-01 00:13:22|         264|         264|   NULL|                B00013|1569913889|1569914002|INTERVAL '0 00:01...|
|              B00014|2019-10-01 00:11:43|2019-10-01 00:37:20|         264|         264|   NULL|                B00014|1569913903|1569915440|INTERVAL '0 00:25...|
|              B00014|

           duration_seconds
0       26298 days 00:30:00
1       26298 days 00:30:00
2        3653 days 00:26:27
3        2922 days 00:01:41
4         366 days 10:00:00
...                     ...
1897488     0 days 00:00:01
1897489     0 days 00:00:01
1897490     0 days 00:00:01
1897491     0 days 00:00:01
1897492     0 days 00:00:01

[1897493 rows x 1 columns]


In [19]:
df = spark.read \
    .option("header", "true") \
    .csv('fhv_tripdata_2019-10.csv')

In [29]:
from pyspark.sql.functions import col

# Assuming your DataFrame is named 'df' and the column is named 'number_column'

# Group by the 'number_column' and count the number of occurrences of each value
count_df = df.groupBy("PUlocationID").count()
count_df = count_df.orderBy(col('count'))
count_df.show()
# Find the least counted number
# least_counted_number = count_df.orderBy("count").first()["PUlocationID"]

# Display the least counted number
# print("Least counted number:", least_counted_number)


+------------+-----+
|PUlocationID|count|
+------------+-----+
|           2|    1|
|         105|    2|
|         111|    5|
|          30|    8|
|         120|   14|
|          12|   15|
|         207|   23|
|          27|   25|
|         154|   26|
|           8|   29|
|         128|   39|
|         253|   47|
|          96|   53|
|          34|   57|
|          59|   62|
|          58|   77|
|          99|   89|
|         190|   98|
|          54|  105|
|         217|  110|
+------------+-----+
only showing top 20 rows



In [ ]:
# Following may needs kernel restart 

In [7]:
import pandas as pd

df_pandas = pd.read_csv('customers-100.csv')

spark.createDataFrame(df_pandas).show()

+-----+---------------+----------+---------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+
|Index|    Customer Id|First Name|Last Name|             Company|             City|             Country|             Phone 1|             Phone 2|               Email|Subscription Date|             Website|
+-----+---------------+----------+---------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+
|    1|DD37Cf93aecA6Dc|    Sheryl|   Baxter|     Rasmussen Group|     East Leonard|               Chile|        229.077.5154|    397.884.0519x718|zunigavanessa@smi...|       2020-08-24|http://www.stephe...|
|    2|1Ef7b82A4CAAD10|   Preston|   Lozano|         Vega-Gentry|East Jimmychester|            Djibouti|          5153435776|    686-620-1820x944|     vmata@colon.com|     

In [9]:
spark.createDataFrame(df_pandas).schema

StructType([StructField('Index', LongType(), True), StructField('Customer Id', StringType(), True), StructField('First Name', StringType(), True), StructField('Last Name', StringType(), True), StructField('Company', StringType(), True), StructField('City', StringType(), True), StructField('Country', StringType(), True), StructField('Phone 1', StringType(), True), StructField('Phone 2', StringType(), True), StructField('Email', StringType(), True), StructField('Subscription Date', StringType(), True), StructField('Website', StringType(), True)])

In [5]:
from pyspark.sql import types

In [16]:
schema = types.StructType([
        types.StructField('Index', types.LongType(),True), 
        types.StructField('Customer Id', types.StringType(), True),
        types.StructField('First Name', types.StringType(), True),
        types.StructField('Last Name',types. StringType(), True),
        types.StructField('Company', types.StringType(), True),
        types.StructField('City', types.StringType(), True),
        types.StructField('Country', types.StringType(), True),
        types.StructField('Phone 1', types.StringType(), True),
        types.StructField('Phone 2', types.StringType(), True),
        types.StructField('Email', types.StringType(), True),
        types.StructField('Subscription Date', types.StringType(), True),
        types.StructField('Website', types.StringType(), True)
                          ])

In [43]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .parquet('fhv_tripdata_october')

In [19]:
df.head(1)

[Row(Index=1, Customer Id='DD37Cf93aecA6Dc', First Name='Sheryl', Last Name='Baxter', Company='Rasmussen Group', City='East Leonard', Country='Chile', Phone 1='229.077.5154', Phone 2='397.884.0519x718', Email='zunigavanessa@smith.info', Subscription Date='2020-08-24', Website='http://www.stephenson.com/')]

In [20]:
df = df.repartition(24)

In [21]:
df.write.parquet('customers/')

24/02/29 00:11:46 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/02/29 00:11:46 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/02/29 00:11:46 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
24/02/29 00:11:46 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
24/02/29 00:11:46 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
24/02/29 00:11:47 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
24/02/29 00:11:47 WARN MemoryManager: Total allocation exceeds 95.

In [7]:
df = spark.read.parquet('customers')

In [8]:
df

DataFrame[Index: bigint, Customer Id: string, First Name: string, Last Name: string, Company: string, City: string, Country: string, Phone 1: string, Phone 2: string, Email: string, Subscription Date: string, Website: string]

In [9]:
df.show()

+-----+---------------+----------+---------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+
|Index|    Customer Id|First Name|Last Name|             Company|               City|             Country|             Phone 1|             Phone 2|               Email|Subscription Date|             Website|
+-----+---------------+----------+---------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+
|   27|aBdfcF2c50b0bfD|    Dakota|    Poole|       Simmons Group|       Michealshire|             Belarus|  (371)987-8576x4720|        071-152-1376| stacey67@fields.org|       2022-02-20|https://sanford-w...|
|   87|ebe77E5Bf9476CE|     Duane|    Woods|      Montoya-Miller|          Lyonsberg|            Maldives|  (636)544-7783x7288|  (203)287-1003x5932|  kmercer@wagner

In [10]:
df.printSchema()

root
 |-- Index: long (nullable = true)
 |-- Customer Id: string (nullable = true)
 |-- First Name: string (nullable = true)
 |-- Last Name: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Phone 1: string (nullable = true)
 |-- Phone 2: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- Subscription Date: string (nullable = true)
 |-- Website: string (nullable = true)



In [21]:
df.select('First Name', 'last Name').filter(df."Customer Id" == 2)

SyntaxError: invalid syntax (2531532772.py, line 1)

In [16]:
df.show(1)

+-----+---------------+----------+---------+-------------+------------+-------+------------------+------------+-------------------+-----------------+--------------------+
|Index|    Customer Id|First Name|Last Name|      Company|        City|Country|           Phone 1|     Phone 2|              Email|Subscription Date|             Website|
+-----+---------------+----------+---------+-------------+------------+-------+------------------+------------+-------------------+-----------------+--------------------+
|   27|aBdfcF2c50b0bfD|    Dakota|    Poole|Simmons Group|Michealshire|Belarus|(371)987-8576x4720|071-152-1376|stacey67@fields.org|       2022-02-20|https://sanford-w...|
+-----+---------------+----------+---------+-------------+------------+-------+------------------+------------+-------------------+-----------------+--------------------+
only showing top 1 row



In [1]:
spark.version

NameError: name 'spark' is not defined